In [1]:
import os
import tensorflow as tf
from PIL import Image
import numpy as np

def get_file(file_dir):
    images = []
    temp = []
    labels = []
    i = 0
    for root,dirs,files in os.walk(file_dir):
        for name in files:
            images.append(os.path.join(root,name))#将路径添加到images中 第一个E:\Lucas_up\Alexnet\image_set\ant        
        for name in dirs:
            temp.append(os.path.join(root,name))  #比image更下一层 第一个E:\Lucas_up\Alexnet\image_set\ant\image_0001.jpg
    for one_floder in temp:
        n_img = len(os.listdir(one_floder))#每一个类的图片有多少张
        labels = np.append(labels,n_img*[i])#建立标签矩阵：每一种图片的数量n_img乘以矩阵[i]append到后面
        i = i + 1

    temp = np.array([images , labels]) #temp.shape = (2,711)
    temp  = temp.transpose()#转置
    np.random.shuffle(temp)             #随机,是真的可以随机...
    image_list = list(temp[:,0])        #路径
    label_list = list(temp[:,1])#现在是一个list而不是矩阵#2.0   numpy.str
    label_list = [int(float(n)) for n in label_list]
    return image_list,label_list
#写一个if判断语句，如果不是jpg文件，则跳过

In [11]:
file_dir = "P:\\data\\GearDataRaw\\GearDataRaw"
images,labels = get_file(file_dir)
print (image_dir)
print (label)

['P:\\data\\GearDataRaw\\GearDataRaw\\5chip5a\\511.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\5chip5a\\440.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\4spall\\337.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\3crack\\254.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\3crack\\291.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\6chip4a\\525.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\3crack\\289.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\4spall\\414.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\2missing\\154.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\5chip5a\\478.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\7chip3a\\656.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\6chip4a\\530.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\5chip5a\\499.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\5chip5a\\460.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\4spall\\354.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\1healthy\\83.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\3crack\\213.jpg', 'P:\\data\\GearDataRaw\\GearDataRaw\\3crack\\241.jpg'

In [14]:
def convert_to_tfrecord(images,labels,save_dir,name):
    filename = os.path.join(save_dir,name + '.tfrecord')
    n_samples = len(labels)
    if np.shape(images)[0] != n_samples:
        raise ValueError('Image size %d does not match label size %d.'%(images.shape,n_samples.shape))
    writer  = tf.io.TFRecordWriter(filename)
    print('\nTransform start....')
    for i in np.arange(n_samples):
        try:
            image = Image.open(images[i])
            image = image.resize((227,227))
            image = np.array(image)
            if image.shape != (227,227,3):
                print(images[i])
                print(labels[i])
                #os.remove(images[i])  # 去除掉所有的非rgb图
            image_raw = image.tostring()
            label = int(labels[i])
            example = tf.train.Example(features = tf.train.Features(
                    feature = {"label":tf.train.Feature(int64_list = tf.train.Int64List(value = [label])),       
                               "img_raw":tf.train.Feature(bytes_list = tf.train.BytesList(value = [image_raw]))}
                    ))
            writer.write(example.SerializeToString())
        except IOError as e:
            print('Could not read:',images[i])
            print('error : %s' %e)
            print('Skip it! \n')
    writer.close()
    print("Transform done!")  

In [30]:
save_dir = 'C:\\Users\\yaz19023\\Pythonoutput\\GearDataRaw\\'
name = 'mytf'
convert_to_tfrecord(images,labels,save_dir,name)


Transform start....
Could not read: P:\data\GearDataRaw\GearDataRaw\1healthy\Thumbs.db
error : cannot identify image file 'P:\\data\\GearDataRaw\\GearDataRaw\\1healthy\\Thumbs.db'
Skip it! 

Could not read: P:\data\GearDataRaw\GearDataRaw\2missing\Thumbs.db
error : cannot identify image file 'P:\\data\\GearDataRaw\\GearDataRaw\\2missing\\Thumbs.db'
Skip it! 

Transform done!


In [28]:
def read_and_decode(tfrecords_file,batch_size):
    filename_queue = tf.data.Dataset.list_files([tfrecords_file])
#     tf.train.string_input_producer([tfrecords_file])
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(filename_queue)
    img_features = tf.parse_single_example(serialized_example,
                                          features={
                                                'label':tf.FixedLenFeature([],tf.int64),
                                              'img_raw':tf.FixedLenFeature([],tf.string),
                                          })
    image = tf.decode_raw(img_features['img_raw'],tf.uint8)
    
    image = tf.reshape(image,[227,227,3])
    
    image = tf.cast(image, tf.float32) * (1. / 255)  #在流中抛出img张量
    label = tf.cast(img_features['label'],tf.int32)
    
    
    image_batch,label_batch = tf.train.batch([image,label],
                                             batch_size=batch_size,
                                             num_threads = 2,
                                             capacity=32)
    label_batch = tf.reshape(label_batch,[batch_size])
    print("Read tfrecord doc done!")
    return image_batch,label_batch

In [29]:
read_and_decode('mytf.tfrecord',20)

AttributeError: module 'tensorflow' has no attribute 'TFRecordReader'